# Many Models Forecasting (MMF)
This notebook showcases how to run MMF with local models on multiple univariate time series of weekly resolution. We will use [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128#sec5) data.

### Cluster setup

We recommend using a cluster with [Databricks Runtime 16.4 LTS for ML](https://docs.databricks.com/en/release-notes/runtime/15.6lts-ml.html). The cluster can be either a single-node or multi-node CPU cluster. MMF leverages [Pandas UDF](https://docs.databricks.com/en/udf/pandas.html) under the hood and utilizes all the available resource. Make sure to set the following Spark configurations before you start your cluster: [`spark.sql.execution.arrow.enabled true`](https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html#enabling-for-conversion-tofrom-pandas) and [`spark.sql.adaptive.enabled false`](https://spark.apache.org/docs/latest/sql-performance-tuning.html#adaptive-query-execution). You can do this by specifying [Spark configuration](https://docs.databricks.com/en/compute/configure.html#spark-configuration) in the advanced options on the cluster creation page.

### Install and import packages
Check out [requirements.txt](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/requirements.txt) if you're interested in the libraries we use.

In [0]:
%pip install -r ../../requirements.txt --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
logging.getLogger("py4j.clientserver").setLevel(logging.ERROR)

In [0]:
import pathlib
import pandas as pd
from datasetsforecast.m4 import M4
from mmf_sa import run_forecast

### Prepare data 
We are using [`datasetsforecast`](https://github.com/Nixtla/datasetsforecast/tree/main/) package to download M4 data. M4 dataset contains a set of time series which we use for testing MMF. Below we have written a number of custome functions to convert M4 time series to an expected format.

In [0]:
# Number of time series
n = 1000


def create_m4_weekly():
    y_df, _, _ = M4.load(directory=str(pathlib.Path.home()), group="Weekly")
    _ids = [f"W{i}" for i in range(1, n)]
    y_df = (
        y_df.groupby("unique_id")
        .filter(lambda x: x.unique_id.iloc[0] in _ids)
        .groupby("unique_id")
        .apply(transform_group)
        .reset_index(drop=True)
    )
    return y_df


def transform_group(df):
    unique_id = df.unique_id.iloc[0]
    if len(df) > 260:
        df = df.iloc[-260:]
    _start = pd.Timestamp("2020-01-01")
    _end = _start + pd.DateOffset(days=int(7*len(df)))
    date_idx = pd.date_range(start=_start, end=_end, freq="W", name="ds")
    res_df = pd.DataFrame(data=[], index=date_idx).reset_index()
    res_df["unique_id"] = unique_id
    res_df["y"] = df.y.values
    return res_df

We are going to save this data in a delta lake table. Provide catalog and database names where you want to store the data.

In [0]:
catalog = "mmf" # Name of the catalog we use to manage our assets
db = "m4" # Name of the schema we use to manage our assets (e.g. datasets)
user = spark.sql('select current_user() as user').collect()[0]['user'] # User email address

In [0]:
# Making sure that the catalog and the schema exist
_ = spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
_ = spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{db}")

(
    spark.createDataFrame(create_m4_weekly())
    .write.format("delta").mode("overwrite")
    .saveAsTable(f"{catalog}.{db}.m4_weekly_train")
)

4.02MiB [00:00, 85.4MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Weekly-train.csv, 4015067, bytes.
44.2kiB [00:00, 50.1MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Weekly-test.csv, 44247, bytes.
4.34MiB [00:00, 114MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded M4-info.csv, 4335598, bytes.
100%|██████████| 3.56M/3.56M [00:00<00:00, 47.5MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded submission-Naive2.zip, 3564691, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed /root/m4/datasets/submission-Naive2.zip


Let's take a peak at the dataset:

In [0]:
display(
  spark.sql(f"select * from {catalog}.{db}.m4_weekly_train where unique_id in ('W1', 'W2', 'W3', 'W4', 'W5') order by unique_id, ds")
  )

ds,unique_id,y
2020-01-05T00:00:00Z,W1,26303.13
2020-01-12T00:00:00Z,W1,26303.13
2020-01-19T00:00:00Z,W1,26594.17
2020-01-26T00:00:00Z,W1,26594.17
2020-02-02T00:00:00Z,W1,27152.07
2020-02-09T00:00:00Z,W1,27152.07
2020-02-16T00:00:00Z,W1,26928.6
2020-02-23T00:00:00Z,W1,26928.6
2020-03-01T00:00:00Z,W1,26489.97
2020-03-08T00:00:00Z,W1,26489.97


If the number of time series is larger than the number of total cores, we set `spark.sql.shuffle.partitions` to the number of cores (can also be a multiple) so that we don't under-utilize the resource.

In [0]:
if n > sc.defaultParallelism:
    sqlContext.setConf("spark.sql.shuffle.partitions", sc.defaultParallelism)

### Models
Let's configure a list of models we are going to apply to our time series for evaluation and forecasting. A comprehensive list of all supported models is available in [mmf_sa/models/README.md](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/README.md). Look for the models where `model_type: local`; these are the local models we import from [statsforecast](https://github.com/Nixtla/statsforecast) and [sktime](https://github.com/sktime/sktime). Check their documentations for the description of each model. 

In [0]:
active_models = [
    "StatsForecastBaselineWindowAverage",
    "StatsForecastBaselineSeasonalWindowAverage",
    "StatsForecastBaselineNaive",
    "StatsForecastBaselineSeasonalNaive",
    "StatsForecastAutoArima",
    "StatsForecastAutoETS",
    "StatsForecastAutoCES",
    "StatsForecastAutoTheta",
    "StatsForecastAutoTbats",
    "StatsForecastAutoMfles",
    "StatsForecastTSB",
    "StatsForecastADIDA",
    "StatsForecastIMAPA",
    "StatsForecastCrostonClassic",
    "StatsForecastCrostonOptimized",
    "StatsForecastCrostonSBA",
    "SKTimeProphet",
    ]

### Run MMF

Now, we can run the evaluation and forecasting using `run_forecast` function defined in [mmf_sa/models/__init__.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/__init__.py). Make sure to set `freq="W"` in `run_forecast` function.

In [0]:
run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_output=f"{catalog}.{db}.weekly_scoring_output",
    evaluation_output=f"{catalog}.{db}.weekly_evaluation_output",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="W",
    prediction_length=4,
    backtest_length=12,
    stride=1,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=True,
    resample=False,
    active_models=active_models,
    experiment_path=f"/Users/{user}/mmf/m4_weekly",
    use_case_name="m4_weekly",
)

Run quality checks


INFO:mmf_sa.data_quality_checks:Starting data quality checks...
INFO:mmf_sa.data_quality_checks:Initial dataset: 82485 records across 359 groups
INFO:mmf_sa.data_quality_checks:Running mandatory configuration checks...
INFO:mmf_sa.data_quality_checks:Running optional data quality checks...
INFO:mmf_sa.data_quality_checks:Data quality summary:
INFO:mmf_sa.data_quality_checks:  - Initial groups: 359
INFO:mmf_sa.data_quality_checks:  - Final groups: 294
INFO:mmf_sa.data_quality_checks:  - Removed groups: 65 (18.1%)
INFO:mmf_sa.data_quality_checks:  - Removal reasons:
INFO:mmf_sa.data_quality_checks:    * missing entries detected and resampling disabled: 65 (100.0%)
INFO:mmf_sa.data_quality_checks:Data quality checks completed successfully. Final dataset: 76440 records across 294 groups


Finished quality checks
Starting evaluate_score
Starting evaluate_models
Started evaluating StatsForecastBaselineWindowAverage
  metric_name  metric_value
0       smape      0.069588
Finished evaluating StatsForecastBaselineWindowAverage
Started evaluating StatsForecastBaselineSeasonalWindowAverage
  metric_name  metric_value
0       smape      0.111473
Finished evaluating StatsForecastBaselineSeasonalWindowAverage
Started evaluating StatsForecastBaselineNaive
  metric_name  metric_value
0       smape      0.057554
Finished evaluating StatsForecastBaselineNaive
Started evaluating StatsForecastBaselineSeasonalNaive
  metric_name  metric_value
0       smape      0.091091
Finished evaluating StatsForecastBaselineSeasonalNaive
Started evaluating StatsForecastAutoArima
  metric_name  metric_value
0       smape      0.052477
Finished evaluating StatsForecastAutoArima
Started evaluating StatsForecastAutoETS
  metric_name  metric_value
0       smape      0.054806
Finished evaluating StatsForec

'6fc9c708-4816-4fbd-8e2e-744f48d3b5d0'

### Evaluate
In `evaluation_output` table, the we store all evaluation results for all backtesting trials from all models.

In [0]:
display(
  spark.sql(f"""
    select * from {catalog}.{db}.weekly_evaluation_output 
    where unique_id = 'W1'
    order by unique_id, model, backtest_window_start_date
    """))

unique_id,backtest_window_start_date,metric_name,metric_value,forecast,actual,model_pickle,run_id,run_date,model,use_case,model_uri
W1,2024-10-06T00:00:00Z,smape,0.027051598965715004,"List(37161.148149555855, 37147.82408476277, 36999.992838642676, 36751.182039004045)","List(35808.59, 35808.59, 36246.14, 36246.14)",gAWVIp0AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-10-13T00:00:00Z,smape,0.015696833255387597,"List(36993.60150008033, 36899.90013299758, 36692.54544737955, 36407.9598053161)","List(35808.59, 36246.14, 36246.14, 36403.7)",gAWVmZ0AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-10-20T00:00:00Z,smape,0.008609242380662116,"List(36694.54092251197, 36519.33736594416, 36261.23503311414, 36015.824955887394)","List(36246.14, 36246.14, 36403.7, 36403.7)",gAWVEJ4AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-10-27T00:00:00Z,smape,0.007136055356698574,"List(36471.56215341396, 36234.05593995193, 36005.73415231325, 35909.47614778921)","List(36246.14, 36403.7, 36403.7, 36150.2)",gAWVg54AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-11-03T00:00:00Z,smape,0.006782873602294534,"List(36227.77079523972, 36008.05268275558, 35920.25426806741, 35970.68345781895)","List(36403.7, 36403.7, 36150.2, 36150.2)",gAWV+p4AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-11-10T00:00:00Z,smape,0.0068564461775906796,"List(36022.09070957928, 35924.91919155233, 35968.62773653715, 35991.58317323432)","List(36403.7, 36150.2, 36150.2, 35790.55)",gAWVdZ8AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-11-17T00:00:00Z,smape,0.003991822016720651,"List(35952.33745367341, 35974.42016408839, 35982.25558850455, 35799.9489017086)","List(36150.2, 36150.2, 35790.55, 35790.55)",gAWV7J8AAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-11-24T00:00:00Z,smape,0.013175130281879499,"List(36023.20143089195, 36026.37409166794, 35843.58219983306, 35497.6592310892)","List(36150.2, 35790.55, 35790.55, 34066.95)",gAWVY6AAAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,SKTimeProphet,m4_weekly,
W1,2024-12-01T00:00:00Z,smape,0.0206600025026252,"List(36031.7685050037, 35841.94891907382, 35491.148054530735, 35228.97200062148)","List(35790.55, 35790.55, 34066.95, 34066.95)",gAWV4aAAAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXS

### Forecast
In `scoring_output` table, forecasts for each time series from each model are stored.

In [0]:
display(spark.sql(f"""
                  select * from {catalog}.{db}.weekly_scoring_output 
                  where unique_id = 'W1'
                  order by unique_id, model, ds
                  """))

unique_id,ds,y,model_pickle,run_id,run_date,use_case,model,model_uri
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34971.86024246254, 35345.799661733836, 35617.0208811529, 35539.15291751882)",gAWVvaIAAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBV5SMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,SKTimeProphet,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(35014.13338454066, 35014.13338454066, 35014.13338454066, 35014.13338454066)",gAWVIx8AAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastADIDA,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34021.60688550331, 33976.26377100662, 33930.92065650993, 33885.57754201324)",gAWVQS4AAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvQVJJTUGUk5QpgZR9lCiMAWSUTowBRJROjAVtYXhfcJQ= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoArima,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(33923.704597473145, 33935.71188354492, 34045.22652053833, 34028.56371688843)",gAWVa0MAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAdBdXRvQ0VTlJOUKYGUfZQojA1zZWFzb25fbGVuZ3RolEsHjAU= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoCES,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34066.95285527504, 34066.95285527504, 34066.95285527504, 34066.95285527504)",gAWVmTkAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAdBdXRvRVRTlJOUKYGUfZQojA1zZWFzb25fbGVuZ3RolEsHjAU= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoETS,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34128.57870152844, 34162.575819287435, 34196.572937046425, 34230.57005480542)",gAWVIVQAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvTUZMRVOUk5QpgZR9lCiMDXNlYXNvbl9sZW5ndGiUSwc= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoMfles,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34259.54559916688, 34142.582351209785, 34073.12652374324, 34124.921030700694)",gAWVQlwAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvVEJBVFOUk5QpgZR9lCiMDXNlYXNvbl9sZW5ndGiUXZQ= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoTbats,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(33928.28484302236, 33939.167033059086, 34037.359194456505, 34039.8602289356)",gAWViD4AAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvVGhldGGUk5QpgZR9lCiMDXNlYXNvbl9sZW5ndGiUSwc= (truncated),6fc9c708-4816-4fbd-8e2e-744f48d3b5d0,2025-07-23T03:45:29.988408Z,m4_weekly,StatsForecastAutoTheta,
W1,"List(2024-12-29T00:00:00Z, 2025-01-05T00:00:00Z, 2025-01-12T00:00:00Z, 2025-01-19T00:00:00Z)","List(34066.95, 34066.95, 34066.95, 34066.95)",gAWVLx8AAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMB

Refer to the [notebook](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/examples/post-evaluation-analysis.ipynb) for guidance on performing fine-grained model selection after running `run_forecast`.

### Delete Tables
Let's clean up the tables.

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.weekly_evaluation_output"))

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.weekly_scoring_output"))